In [ ]:
import numpy as np
import time
import math
import plotly.graph_objects as go  
from simpleai.search import astar, SearchProblem
from simpleai.search import breadth_first, greedy, uniform_cost, depth_first, hill_climbing
import webbrowser
import plotly.io as pio
import os

class MarsRouteProblem(SearchProblem):
    def __init__(self, mars_map, start, goal, max_height_diff=0.75):
        """
        Initialize the Mars route planning problem.
        
        :param mars_map: NumPy array of elevation data
        :param start: Starting (x, y) coordinate
        :param goal: Goal (x, y) coordinate
        :param max_height_diff: Maximum allowed height difference between adjacent pixels
        """
        self.mars_map = mars_map
        self.scale = 10.0177  #meters per pixel
        self.nr, self.nc = mars_map.shape
        
        # Convert geographic coordinates to matrix indices
        self.start_row = self.nr - round(start[1] / self.scale)
        self.start_col = round(start[0] / self.scale)
        
        self.goal_row = self.nr - round(goal[1] / self.scale)
        self.goal_col = round(goal[0] / self.scale)
        
        self.max_height_diff = max_height_diff
        
        # Validate start and goal points
        self.validate_point(self.start_row, self.start_col, "Start")
        self.validate_point(self.goal_row, self.goal_col, "Goal")
        
        SearchProblem.__init__(self, (self.start_row, self.start_col))
        
    def validate_point(self, row, col, point_type):
        """Validate that a point is within map bounds and has valid elevation."""
        if (row < 0 or row >= self.nr or 
            col < 0 or col >= self.nc or 
            self.mars_map[row, col] == -1):
            raise ValueError(f"{point_type} point is invalid: ({row}, {col})")
    
    def actions(self, state):
        """Determine valid adjacent moves."""
        row, col = state
        actions = []
        
        # 8-directional movement (including diagonals)
        directions = [
            (-1, -1), (-1, 0), (-1, 1),
            (0, -1),           (0, 1),
            (1, -1), (1, 0), (1, 1)
        ]
        
        for dr, dc in directions:
            new_row, new_col = row + dr, col + dc
            
            # Check bounds and elevation validity
            if (0 <= new_row < self.nr and 
                0 <= new_col < self.nc and 
                self.mars_map[new_row, new_col] != -1):
                
                # Check height difference
                current_height = self.mars_map[row, col]
                new_height = self.mars_map[new_row, new_col]
                
                if abs(current_height - new_height) <= self.max_height_diff:
                    actions.append((dr, dc))
        
        return actions
    
    def result(self, state, action):
        """Apply the action to the current state."""
        row, col = state
        dr, dc = action
        return (row + dr, col + dc)
    
    def is_goal(self, state):
        """Check if current state is the goal state."""
        return state == (self.goal_row, self.goal_col)
    
    def cost(self, state, action, state2):
        """Calculate movement cost."""
        row, col = state
        new_row, new_col = state2
        
        # Euclidean distance
        return math.sqrt((new_row - row)**2 + (new_col - col)**2)
    
    def heuristic(self, state):
        """Calculate heuristic using Euclidean distance."""
        row, col = state
        return math.sqrt((row - self.goal_row)**2 + (col - self.goal_col)**2)
    
    #FUNCTION FOR HILL CLIMBING
    def value(self, state):
    #negative heuristic to maximize the objective's closeness
        return -self.heuristic(state)

def visualize_3d_route(mars_map, start, goal, path):
    """
    Create a 3D visualization of the route on the Mars height map.
    
    :param mars_map: NumPy array of elevation data
    :param start: (x, y) start coordinate
    :param goal: (x, y) goal coordinate
    :param path: List of (row, col) coordinates in the path
    """
    # Prepare map coordinates
    scale = 10.0177
    x = scale * np.arange(mars_map.shape[1])
    y = scale * np.arange(mars_map.shape[0])
    X, Y = np.meshgrid(x, y)
    
    # Prepare path coordinates
    nr, nc = mars_map.shape
    path_x = []
    path_y = []
    path_z = []
    
    for row, col in path:
        # Convert back to geographic coordinates
        px = col * scale
        py = (nr - row) * scale
        pz = mars_map[row, col]
        
        path_x.append(px)
        path_y.append(py)
        path_z.append(pz)
    
    # Create 3D visualization
    fig = go.Figure(
        data=[
            go.Surface(
                x=X, y=Y, z=np.flipud(mars_map), 
                colorscale='hot', 
                cmin=0, 
                lighting=dict(
                    ambient=0.0, 
                    diffuse=0.8, 
                    fresnel=0.02, 
                    roughness=0.4, 
                    specular=0.2
                ), 
                lightposition=dict(
                    x=0, 
                    y=nr/2, 
                    z=2*mars_map.max()
                )
            ),
            go.Scatter3d(
                x=path_x, 
                y=path_y, 
                z=path_z, 
                name='path', 
                mode='lines+markers', 
                line=dict(color='red', width=5),
                marker=dict(
                    color=np.linspace(0, 1, len(path_x)), 
                    colorscale="Plotly3", 
                    size=4
                )
            )
        ]
    )
    
    # Update layout
    fig.update_layout(
        scene_aspectmode='manual', 
        scene_aspectratio=dict(
            x=1, 
            y=nr/nc, 
            z=max(mars_map.max()/x.max(), 0.2)
        ), 
        scene_zaxis_range=[0, mars_map.max()]
    )
    
    return fig

def solve_mars_route(mars_map, start, goal, max_height_diff=0.75):
    """
    Solve route planning problem using A* search.
    
    :param mars_map: NumPy array of elevation data
    :param start: (x, y) start coordinate
    :param goal: (x, y) goal coordinate
    :param max_height_diff: Maximum allowed height difference
    :return: Route details
    """
    start_time = time.time()
    
    # Create problem instance
    problem = MarsRouteProblem(mars_map, start, goal, max_height_diff)
    
    #-----------------ALGORITHMS --------------------
    #A* search
    #result = astar(problem, graph_search=True)
    #BFS
    #result = breadth_first(problem, graph_search=True)
    #Greedy
    #result = greedy(problem, graph_search=True)
    #Uniform Cost
    #result = uniform_cost(problem, graph_search=True)
    #Hill Climbing
    result = hill_climbing(problem)
    end_time = time.time()
    
    if not result:
        return None
    
    # Extract path
    path = [state for _, state in result.path()]
    
    # Calculate total distance
    total_distance = 0
    for i in range(1, len(path)):
        row1, col1 = path[i-1]
        row2, col2 = path[i]
        total_distance += math.sqrt((row1 - row2)**2 + (col1 - col2)**2) * 10.0177
    
    return {
        'path': path,
        'distance': total_distance,
        'time': end_time - start_time,
        'route_figure': visualize_3d_route(mars_map, start, goal, path)
    }

def main():
    # Load Mars map
    mars_map = np.load('/Users/ximenacanton/Documents/TEC/4 SEM/DAI/mars/map.npy')
    
    # Routes to test
    routes = [
        ((5000, 7600), (3600, 8600)),
        ((1400, 3600), (1900, 5800)),
        ((6050, 1750), (4600, 2500)),
        ((2500, 6000), (2200, 9000)),
        ((4300, 4700), (730, 9200)),
        ((2300, 1370), (4000, 11500))
    ]
    
    # Solve and plot each route
    for i, (start, goal) in enumerate(routes, 1):
        print(f"\nRoute {i}: From {start} to {goal}")
        
        result = solve_mars_route(mars_map, start, goal)
        
        if result:
            print(f"Distance traveled: {result['distance']:.2f} meters")
            print(f"Route calculation time: {result['time']:.4f} seconds")
            
            # Definir la ruta del archivo HTML
            file_path = f'route_{i}_map.html'

            # Guardar la figura como HTML
            result['route_figure'].write_html(file_path)

            # Abrir automáticamente el archivo en el navegador
            pio.write_html(result['route_figure'], file_path, auto_open=True)  
            webbrowser.open(f"file://{os.path.abspath(file_path)}")  # Abrir manualmente con ruta absoluta

        else:
            print("No route found!")

if __name__ == "__main__":
    main()


Route 1: From (5000, 7600) to (3600, 8600)
Distance traveled: 0.00 meters
Route calculation time: 0.0003 seconds

Route 2: From (1400, 3600) to (1900, 5800)
Distance traveled: 0.00 meters
Route calculation time: 0.0013 seconds

Route 3: From (6050, 1750) to (4600, 2500)
Distance traveled: 0.00 meters
Route calculation time: 0.0011 seconds

Route 4: From (2500, 6000) to (2200, 9000)
Distance traveled: 0.00 meters
Route calculation time: 0.0560 seconds

Route 5: From (4300, 4700) to (730, 9200)
Distance traveled: 0.00 meters
Route calculation time: 0.0509 seconds

Route 6: From (2300, 1370) to (4000, 11500)
Distance traveled: 0.00 meters
Route calculation time: 0.0453 seconds
